<a href="https://colab.research.google.com/github/Jay22519/CandleSticks/blob/main/candlestick_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Colab!

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=Warning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# IMPORTS
import pandas as pd
import numpy as np

import time
import math
import os.path

# from tqdm import tnrange, notebook
from tqdm.notebook import tqdm

from datetime import timedelta, datetime
from dateutil import parser

# Import the plotting library
import matplotlib.pyplot as plt
# %matplotlib inline

import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib.dates import MonthLocator

import seaborn as sns
sns.set()

plt.rcParams.update({'figure.figsize':(15,7), 'figure.dpi':120})
# plt.style.use('ggplot')

### Step 1: Set up Kaggle API Credentials

1.  **Generate Kaggle API Token:**
    *   Go to your Kaggle account page: `https://www.kaggle.com/<your-username>/account`
    *   Scroll down to the 'API' section.
    *   Click on 'Create New API Token'. This will download a `kaggle.json` file to your computer.

2.  **Upload `kaggle.json` to Colab:**
    *   In Colab, click on the folder icon on the left sidebar (File browser).
    *   Click on the 'Upload' icon.
    *   Select the `kaggle.json` file you just downloaded and upload it to the root directory of your Colab environment (`/content/`).

3.  **Set up permissions (run the following cell):**

In [ ]:
import os

# Set your Kaggle username and key as environment variables
# Replace with your actual Kaggle username
os.environ['KAGGLE_USERNAME'] = 'jaypurigoswami'
# Replace with your actual Kaggle API key
os.environ['KAGGLE_KEY'] = 'KGAT_2389149e7c6562d05f133d6561a218d7'

# Create the .kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle

# Create a kaggle.json file with the credentials
# This method avoids copying a file, directly uses the environment variables
with open(os.path.expanduser('~/.kaggle/kaggle.json'), 'w') as f:
    f.write(f'{{"username":"{os.environ["KAGGLE_USERNAME"]}","key":"{os.environ["KAGGLE_KEY"]}"}}')

# Set permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle API credentials set up successfully!")

Kaggle API credentials set up successfully!


### Step 2: Download the Dataset

Now you can use the Kaggle API to download the dataset. The dataset you mentioned is 'ethereum-eth-7-exchanges-1m-full-historical-data'. You can find its API command on the dataset's Kaggle page under the 'API' section.

Here's an example of how to download a dataset. Please replace `YOUR_KAGGLE_USERNAME/YOUR_DATASET_NAME` with the correct path for your dataset. For example, for 'Ethereum ETH, 7 Exchanges, 1m Full Historical Data', the command might look like `!kaggle datasets download -d mariosparx/ethereum-eth-7-exchanges-1m-full-historical-data`.

After downloading, you'll likely need to unzip the file.

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [ ]:
# Set the path to the file you'd like to load
file_path = "ETHUSD_1m_Binance.csv"

In [ ]:
# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "imranbukhari/comprehensive-ethusd-1m-data",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

100%|██████████| 526M/526M [00:05<00:00, 99.7MB/s]


In [ ]:
df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2017-08-17 04:00:00,301.13,301.13,301.13,301.13,0.42643,2017-08-17 04:00:59.999,128.410866,2.0,0.42643,128.410866,0.0
1,2017-08-17 04:01:00,301.13,301.13,301.13,301.13,2.75787,2017-08-17 04:01:59.999,830.477393,4.0,2.75787,830.477393,0.0
2,2017-08-17 04:02:00,300.00,300.00,300.00,300.00,0.09930,2017-08-17 04:02:59.999,29.790000,2.0,0.09930,29.790000,0.0
3,2017-08-17 04:03:00,300.00,300.00,300.00,300.00,0.31389,2017-08-17 04:03:59.999,94.167000,3.0,0.00000,0.000000,0.0
4,2017-08-17 04:04:00,301.13,301.13,301.13,301.13,0.23202,2017-08-17 04:04:59.999,69.868183,1.0,0.23202,69.868183,0.0


In [ ]:
df.columns

Index(['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
       'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
       'Taker buy quote asset volume', 'Ignore'],
      dtype='object')

In [ ]:
def cleanPx(prices, freq='1H'):
    # Ensure 'Open time' is treated as datetime for setting index
    prices["Open time"] = pd.to_datetime(prices["Open time"])
    prices = prices.iloc[prices["Open time"].drop_duplicates(keep='last').index]

    prices.set_index('Open time', inplace=True)

    prices_ohlc = prices[['Open','High','Low','Close']]
    prices_vol = prices[['Volume']]

    prices_ohlc = prices_ohlc.resample(freq).agg({'Open': 'first',
                                 'High': 'max',
                                 'Low': 'min',
                                 'Close': 'last'})
    prices_vol = prices_vol.resample(freq).sum()

    prices = pd.concat([prices_ohlc, prices_vol], axis=1)
    prices.index = prices.index.tz_localize('UTC').tz_convert('Asia/Singapore')

    return prices.dropna()

In [ ]:
data = cleanPx(df)
display(data.head())

,Open,High,Low,Close,Volume
Open time,,,,,
2017-08-17 12:00:00+08:00,301.13,302.57,298.0,301.61,125.66877
2017-08-17 13:00:00+08:00,301.61,303.28,300.0,303.10,377.67246
2017-08-17 14:00:00+08:00,302.40,304.44,301.9,302.68,303.86672
2017-08-17 15:00:00+08:00,302.68,307.96,302.6,307.96,754.74510
2017-08-17 16:00:00+08:00,307.95,309.97,307.0,308.62,150.75029


In [ ]:
data.reset_index(inplace = True)

In [ ]:
data.columns

Index(['Open time', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [ ]:
data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

In [ ]:
data.head()

,Date,Open,High,Low,Close,Volume
0,2017-08-17 12:00:00+08:00,301.13,302.57,298.0,301.61,125.66877
1,2017-08-17 13:00:00+08:00,301.61,303.28,300.0,303.10,377.67246
2,2017-08-17 14:00:00+08:00,302.40,304.44,301.9,302.68,303.86672
3,2017-08-17 15:00:00+08:00,302.68,307.96,302.6,307.96,754.74510
4,2017-08-17 16:00:00+08:00,307.95,309.97,307.0,308.62,150.75029


In [ ]:
data.set_index('Date', inplace=True)

In [ ]:
import talib
candle_names = talib.get_function_groups()['Pattern Recognition']
removed = ['CDLCOUNTERATTACK', 'CDLLONGLINE', 'CDLSHORTLINE',
           'CDLSTALLEDPATTERN', 'CDLKICKINGBYLENGTH']
candle_names = [name for name in candle_names if name not in removed]

ModuleNotFoundError: No module named 'talib'

In [ ]:
import talib
candle_names = talib.get_function_groups()['Pattern Recognition']
removed = ['CDLCOUNTERATTACK', 'CDLLONGLINE', 'CDLSHORTLINE',
           'CDLSTALLEDPATTERN', 'CDLKICKINGBYLENGTH']
candle_names = [name for name in candle_names if name not in removed]

In [ ]:
!pip install ta-lib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 37.4 MB/s eta 0:00:00


In [ ]:
', '.join(candle_names)

'CDL2CROWS, CDL3BLACKCROWS, CDL3INSIDE, CDL3LINESTRIKE, CDL3OUTSIDE, CDL3STARSINSOUTH, CDL3WHITESOLDIERS, CDLABANDONEDBABY, CDLADVANCEBLOCK, CDLBELTHOLD, CDLBREAKAWAY, CDLCLOSINGMARUBOZU, CDLCONCEALBABYSWALL, CDLDARKCLOUDCOVER, CDLDOJI, CDLDOJISTAR, CDLDRAGONFLYDOJI, CDLENGULFING, CDLEVENINGDOJISTAR, CDLEVENINGSTAR, CDLGAPSIDESIDEWHITE, CDLGRAVESTONEDOJI, CDLHAMMER, CDLHANGINGMAN, CDLHARAMI, CDLHARAMICROSS, CDLHIGHWAVE, CDLHIKKAKE, CDLHIKKAKEMOD, CDLHOMINGPIGEON, CDLIDENTICAL3CROWS, CDLINNECK, CDLINVERTEDHAMMER, CDLKICKING, CDLLADDERBOTTOM, CDLLONGLEGGEDDOJI, CDLMARUBOZU, CDLMATCHINGLOW, CDLMATHOLD, CDLMORNINGDOJISTAR, CDLMORNINGSTAR, CDLONNECK, CDLPIERCING, CDLRICKSHAWMAN, CDLRISEFALL3METHODS, CDLSEPARATINGLINES, CDLSHOOTINGSTAR, CDLSPINNINGTOP, CDLSTICKSANDWICH, CDLTAKURI, CDLTASUKIGAP, CDLTHRUSTING, CDLTRISTAR, CDLUNIQUE3RIVER, CDLUPSIDEGAP2CROWS, CDLXSIDEGAP3METHODS'

In [ ]:
candle_rankings = {
        "CDL3LINESTRIKE_Bull": 1,
        "CDL3LINESTRIKE_Bear": 2,
        "CDL3BLACKCROWS_Bull": 3,
        "CDL3BLACKCROWS_Bear": 3,
        "CDLEVENINGSTAR_Bull": 4,
        "CDLEVENINGSTAR_Bear": 4,
        "CDLTASUKIGAP_Bull": 5,
        "CDLTASUKIGAP_Bear": 5,
        "CDLINVERTEDHAMMER_Bull": 6,
        "CDLINVERTEDHAMMER_Bear": 6,
        "CDLMATCHINGLOW_Bull": 7,
        "CDLMATCHINGLOW_Bear": 7,
        "CDLABANDONEDBABY_Bull": 8,
        "CDLABANDONEDBABY_Bear": 8,
        "CDLBREAKAWAY_Bull": 10,
        "CDLBREAKAWAY_Bear": 10,
        "CDLMORNINGSTAR_Bull": 12,
        "CDLMORNINGSTAR_Bear": 12,
        "CDLPIERCING_Bull": 13,
        "CDLPIERCING_Bear": 13,
        "CDLSTICKSANDWICH_Bull": 14,
        "CDLSTICKSANDWICH_Bear": 14,
        "CDLTHRUSTING_Bull": 15,
        "CDLTHRUSTING_Bear": 15,
        "CDLINNECK_Bull": 17,
        "CDLINNECK_Bear": 17,
        "CDL3INSIDE_Bull": 20,
        "CDL3INSIDE_Bear": 56,
        "CDLHOMINGPIGEON_Bull": 21,
        "CDLHOMINGPIGEON_Bear": 21,
        "CDLDARKCLOUDCOVER_Bull": 22,
        "CDLDARKCLOUDCOVER_Bear": 22,
        "CDLIDENTICAL3CROWS_Bull": 24,
        "CDLIDENTICAL3CROWS_Bear": 24,
        "CDLMORNINGDOJISTAR_Bull": 25,
        "CDLMORNINGDOJISTAR_Bear": 25,
        "CDLXSIDEGAP3METHODS_Bull": 27,
        "CDLXSIDEGAP3METHODS_Bear": 26,
        "CDLTRISTAR_Bull": 28,
        "CDLTRISTAR_Bear": 76,
        "CDLGAPSIDESIDEWHITE_Bull": 46,
        "CDLGAPSIDESIDEWHITE_Bear": 29,
        "CDLEVENINGDOJISTAR_Bull": 30,
        "CDLEVENINGDOJISTAR_Bear": 30,
        "CDL3WHITESOLDIERS_Bull": 32,
        "CDL3WHITESOLDIERS_Bear": 32,
        "CDLONNECK_Bull": 33,
        "CDLONNECK_Bear": 33,
        "CDL3OUTSIDE_Bull": 34,
        "CDL3OUTSIDE_Bear": 39,
        "CDLRICKSHAWMAN_Bull": 35,
        "CDLRICKSHAWMAN_Bear": 35,
        "CDLSEPARATINGLINES_Bull": 36,
        "CDLSEPARATINGLINES_Bear": 40,
        "CDLLONGLEGGEDDOJI_Bull": 37,
        "CDLLONGLEGGEDDOJI_Bear": 37,
        "CDLHARAMI_Bull": 38,
        "CDLHARAMI_Bear": 72,
        "CDLLADDERBOTTOM_Bull": 41,
        "CDLLADDERBOTTOM_Bear": 41,
        "CDLCLOSINGMARUBOZU_Bull": 70,
        "CDLCLOSINGMARUBOZU_Bear": 43,
        "CDLTAKURI_Bull": 47,
        "CDLTAKURI_Bear": 47,
        "CDLDOJISTAR_Bull": 49,
        "CDLDOJISTAR_Bear": 51,
        "CDLHARAMICROSS_Bull": 50,
        "CDLHARAMICROSS_Bear": 80,
        "CDLADVANCEBLOCK_Bull": 54,
        "CDLADVANCEBLOCK_Bear": 54,
        "CDLSHOOTINGSTAR_Bull": 55,
        "CDLSHOOTINGSTAR_Bear": 55,
        "CDLMARUBOZU_Bull": 71,
        "CDLMARUBOZU_Bear": 57,
        "CDLUNIQUE3RIVER_Bull": 60,
        "CDLUNIQUE3RIVER_Bear": 60,
        "CDL2CROWS_Bull": 61,
        "CDL2CROWS_Bear": 61,
        "CDLBELTHOLD_Bull": 62,
        "CDLBELTHOLD_Bear": 63,
        "CDLHAMMER_Bull": 65,
        "CDLHAMMER_Bear": 65,
        "CDLHIGHWAVE_Bull": 67,
        "CDLHIGHWAVE_Bear": 67,
        "CDLSPINNINGTOP_Bull": 69,
        "CDLSPINNINGTOP_Bear": 73,
        "CDLUPSIDEGAP2CROWS_Bull": 74,
        "CDLUPSIDEGAP2CROWS_Bear": 74,
        "CDLGRAVESTONEDOJI_Bull": 77,
        "CDLGRAVESTONEDOJI_Bear": 77,
        "CDLHIKKAKEMOD_Bull": 82,
        "CDLHIKKAKEMOD_Bear": 81,
        "CDLHIKKAKE_Bull": 85,
        "CDLHIKKAKE_Bear": 83,
        "CDLENGULFING_Bull": 84,
        "CDLENGULFING_Bear": 91,
        "CDLMATHOLD_Bull": 86,
        "CDLMATHOLD_Bear": 86,
        "CDLHANGINGMAN_Bull": 87,
        "CDLHANGINGMAN_Bear": 87,
        "CDLRISEFALL3METHODS_Bull": 94,
        "CDLRISEFALL3METHODS_Bear": 89,
        "CDLKICKING_Bull": 96,
        "CDLKICKING_Bear": 102,
        "CDLDRAGONFLYDOJI_Bull": 98,
        "CDLDRAGONFLYDOJI_Bear": 98,
        "CDLCONCEALBABYSWALL_Bull": 101,
        "CDLCONCEALBABYSWALL_Bear": 101,
        "CDL3STARSINSOUTH_Bull": 103,
        "CDL3STARSINSOUTH_Bear": 103,
        "CDLDOJI_Bull": 104,
        "CDLDOJI_Bear": 104
    }


In [ ]:
data.reset_index(inplace=True)
data = data[['Date', 'Open', 'High', 'Low', 'Close']]
data.columns = ['time', 'open', 'high', 'low', 'close']

In [ ]:
data.head()

,time,open,high,low,close
0,2017-08-17 12:00:00+08:00,301.13,302.57,298.0,301.61
1,2017-08-17 13:00:00+08:00,301.61,303.28,300.0,303.10
2,2017-08-17 14:00:00+08:00,302.40,304.44,301.9,302.68
3,2017-08-17 15:00:00+08:00,302.68,307.96,302.6,307.96
4,2017-08-17 16:00:00+08:00,307.95,309.97,307.0,308.62


In [ ]:
op = data['open']
hi = data['high']
lo = data['low']
cl = data['close']

# create columns for each pattern
for candle in candle_names:
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    data[candle] = getattr(talib, candle)(op, hi, lo, cl)

In [ ]:
data.head()

,time,open,high,low,close,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,...,CDLSHOOTINGSTAR,CDLSPINNINGTOP,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS
0,2017-08-17 12:00:00+08:00,301.13,302.57,298.0,301.61,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2017-08-17 13:00:00+08:00,301.61,303.28,300.0,303.10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2017-08-17 14:00:00+08:00,302.40,304.44,301.9,302.68,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2017-08-17 15:00:00+08:00,302.68,307.96,302.6,307.96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2017-08-17 16:00:00+08:00,307.95,309.97,307.0,308.62,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from itertools import compress

data['candlestick_pattern'] = np.nan
data['candlestick_match_count'] = np.nan

for index, row in data.iterrows():

    # no pattern found
    if len(row[candle_names]) - sum(row[candle_names] == 0) == 0:
        data.loc[index,'candlestick_pattern'] = "NO_PATTERN"
        data.loc[index, 'candlestick_match_count'] = 0
    # single pattern found
    elif len(row[candle_names]) - sum(row[candle_names] == 0) == 1:
        # bull pattern 100 or 200
        if any(row[candle_names].values > 0):
            pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bull'
            data.loc[index, 'candlestick_pattern'] = pattern
            data.loc[index, 'candlestick_match_count'] = 1
        # bear pattern -100 or -200
        else:
            pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bear'
            data.loc[index, 'candlestick_pattern'] = pattern
            data.loc[index, 'candlestick_match_count'] = 1
    # multiple patterns matched -- select best performance
    else:
        # filter out pattern names from bool list of values
        patterns = list(compress(row[candle_names].keys(), row[candle_names].values != 0))
        container = []
        for pattern in patterns:
            if row[pattern] > 0:
                container.append(pattern + '_Bull')
            else:
                container.append(pattern + '_Bear')
        rank_list = [candle_rankings[p] for p in container]
        if len(rank_list) == len(container):
            rank_index_best = rank_list.index(min(rank_list))
            data.loc[index, 'candlestick_pattern'] = container[rank_index_best]
            data.loc[index, 'candlestick_match_count'] = len(container)

In [ ]:
data.head()

,time,open,high,low,close,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,...,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS,candlestick_pattern,candlestick_match_count
0,2017-08-17 12:00:00+08:00,301.13,302.57,298.0,301.61,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NO_PATTERN,0.0
1,2017-08-17 13:00:00+08:00,301.61,303.28,300.0,303.10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NO_PATTERN,0.0
2,2017-08-17 14:00:00+08:00,302.40,304.44,301.9,302.68,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NO_PATTERN,0.0
3,2017-08-17 15:00:00+08:00,302.68,307.96,302.6,307.96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NO_PATTERN,0.0
4,2017-08-17 16:00:00+08:00,307.95,309.97,307.0,308.62,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NO_PATTERN,0.0


In [ ]:
data1 = data[['time' , 'open' , 'high' , 'low' , 'close' , 'candlestick_pattern']]

In [ ]:
data1.head()

,time,open,high,low,close,candlestick_pattern
0,2017-08-17 12:00:00+08:00,301.13,302.57,298.0,301.61,NO_PATTERN
1,2017-08-17 13:00:00+08:00,301.61,303.28,300.0,303.10,NO_PATTERN
2,2017-08-17 14:00:00+08:00,302.40,304.44,301.9,302.68,NO_PATTERN
3,2017-08-17 15:00:00+08:00,302.68,307.96,302.6,307.96,NO_PATTERN
4,2017-08-17 16:00:00+08:00,307.95,309.97,307.0,308.62,NO_PATTERN


In [ ]:
data2 = data[data['candlestick_pattern'] != "NO_PATTERN"]

In [ ]:
data2.head()

,time,open,high,low,close,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,...,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS,candlestick_pattern,candlestick_match_count
9,2017-08-17 21:00:00+08:00,308.62,309.72,307.77,309.40,0,0,0,0,0,...,0,0,0,0,0,0,0,0,CDLENGULFING_Bull,2.0
11,2017-08-17 23:00:00+08:00,306.50,308.76,303.56,307.06,0,0,0,0,0,...,0,0,0,0,0,0,0,0,CDLHARAMI_Bull,3.0
12,2017-08-18 00:00:00+08:00,307.06,312.18,298.21,308.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,CDLHIGHWAVE_Bull,2.0
13,2017-08-18 01:00:00+08:00,308.00,308.33,303.98,308.33,0,0,0,0,0,...,0,100,0,0,0,0,0,0,CDLLONGLEGGEDDOJI_Bull,4.0
15,2017-08-18 03:00:00+08:00,309.10,309.84,301.60,301.60,0,0,0,0,0,...,0,0,0,0,0,0,0,0,CDLCLOSINGMARUBOZU_Bear,2.0
